In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [43]:
metadata = pd.read_csv('input/rsna-breast-cancer-detection/vindr_mammo/vindr-mammo-a-large-scale-benchmark-dataset-for-computer-aided-detection-and-diagnosis-in-full-field-digital-mammography-1.0.0/metadata.csv')
breast_anno = pd.read_csv('input/rsna-breast-cancer-detection/vindr_mammo/vindr-mammo-a-large-scale-benchmark-dataset-for-computer-aided-detection-and-diagnosis-in-full-field-digital-mammography-1.0.0/breast-level_annotations.csv')
finding_anno = pd.read_csv('input/rsna-breast-cancer-detection/vindr_mammo/vindr-mammo-a-large-scale-benchmark-dataset-for-computer-aided-detection-and-diagnosis-in-full-field-digital-mammography-1.0.0/finding_annotations.csv')

In [44]:
breast_anno.head()

,study_id,series_id,image_id,laterality,view_position,height,width,breast_birads,breast_density,split
0,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,d8125545210c08e1b1793a5af6458ee2,L,CC,3518,2800,BI-RADS 2,DENSITY C,training
1,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,290c658f4e75a3f83ec78a847414297c,L,MLO,3518,2800,BI-RADS 2,DENSITY C,training
2,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,cd0fc7bc53ac632a11643ac4cc91002a,R,CC,3518,2800,BI-RADS 2,DENSITY C,training
3,b8d273e8601f348d3664778dae0e7e0b,b36517b9cbbcfd286a7ae04f643af97a,71638b1e853799f227492bfb08a01491,R,MLO,3518,2800,BI-RADS 2,DENSITY C,training
4,8269f5971eaca3e5d3772d1796e6bd7a,d931832a0815df082c085b6e09d20aac,dd9ce3288c0773e006a294188aadba8e,L,CC,3518,2800,BI-RADS 1,DENSITY C,training


In [45]:
metadata['Manufacturer'].value_counts()

SIEMENS              15244
Planmed               3796
IMS s.r.l.             736
IMS GIOTTO S.p.A.      224
Name: Manufacturer, dtype: int64

In [46]:
metadata['Manufacturer\'s Model Name'].value_counts()

Mammomat Inspiration    15244
Planmed Nuance           3796
GIOTTO CLASS              628
GIOTTO IMAGE 3DL          332
Name: Manufacturer's Model Name, dtype: int64

In [47]:
breast_anno['breast_birads'].value_counts()

BI-RADS 1    13406
BI-RADS 2     4676
BI-RADS 3      930
BI-RADS 4      762
BI-RADS 5      226
Name: breast_birads, dtype: int64

In [8]:
breast_anno['breast_density'].value_counts()

DENSITY C    15292
DENSITY D     2700
DENSITY B     1908
DENSITY A      100
Name: breast_density, dtype: int64

In [53]:
metadata2 = metadata.rename(
    {'SOP Instance UID.1': 'image_id', 'Patient\'s Age': 'age' , 'Manufacturer\'s Model Name': 'machine_id'}, 
    axis=1)[['image_id', 'age', 'machine_id']]
metadata2['age'] = metadata2['age'].str[:-1].astype(float)
metadata2.head()

,image_id,age,machine_id
0,d8125545210c08e1b1793a5af6458ee2,53.0,Mammomat Inspiration
1,290c658f4e75a3f83ec78a847414297c,53.0,Mammomat Inspiration
2,cd0fc7bc53ac632a11643ac4cc91002a,53.0,Mammomat Inspiration
3,71638b1e853799f227492bfb08a01491,53.0,Mammomat Inspiration
4,dd9ce3288c0773e006a294188aadba8e,42.0,Mammomat Inspiration


In [54]:
breast_anno2 = breast_anno.rename({
    'study_id': 'patient_id', 'view_position': 'view', 'breast_birads': 'BIRADS', 'breast_density': 'density', 'split': 'is_test'
}, axis=1)[['patient_id', 'image_id', 'view', 'laterality', 'view', 'BIRADS', 'density', 'is_test']]
breast_anno2['BIRADS'] = breast_anno2['BIRADS'].str[-1].astype(int)
breast_anno2['density'] = breast_anno2['density'].str[-1]
breast_anno2['is_test'] = (breast_anno2['is_test'] == 'test').astype(int)
breast_anno2.head()

,patient_id,image_id,view,laterality,view,BIRADS,density,is_test
0,b8d273e8601f348d3664778dae0e7e0b,d8125545210c08e1b1793a5af6458ee2,CC,L,CC,2,C,0
1,b8d273e8601f348d3664778dae0e7e0b,290c658f4e75a3f83ec78a847414297c,MLO,L,MLO,2,C,0
2,b8d273e8601f348d3664778dae0e7e0b,cd0fc7bc53ac632a11643ac4cc91002a,CC,R,CC,2,C,0
3,b8d273e8601f348d3664778dae0e7e0b,71638b1e853799f227492bfb08a01491,MLO,R,MLO,2,C,0
4,8269f5971eaca3e5d3772d1796e6bd7a,dd9ce3288c0773e006a294188aadba8e,CC,L,CC,1,C,0


In [64]:
finding_anno2 = finding_anno.rename({
    'study_id': 'patient_id', 'view_position': 'view',
}, axis=1).drop(['patient_id', 'series_id'], axis=1)
finding_anno2.head()

,image_id,laterality,view,height,width,breast_birads,breast_density,finding_categories,finding_birads,xmin,ymin,xmax,ymax,split
0,4e3a578fe535ea4f5258d3f7f4419db8,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Mass'],BI-RADS 4,2355.139893,1731.640015,2482.979980,1852.750000,training
1,dac39351b0f3a8c670b7f8dc88029364,R,MLO,3518,2800,BI-RADS 4,DENSITY C,['Mass'],BI-RADS 4,2386.679932,1240.609985,2501.800049,1354.040039,training
2,c83f780904f25eacb44e9030f32c66e1,R,CC,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],BI-RADS 3,2279.179932,1166.510010,2704.439941,2184.260010,training
3,893528bc38a0362928a89364f1b692fd,R,MLO,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],BI-RADS 3,1954.270020,1443.640015,2589.760010,2193.810059,training
4,318264c881bf12f2c1efe5f93920cc37,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Architectural Distortion'],BI-RADS 4,2172.300049,1967.410034,2388.699951,2147.159912,training


In [57]:
train = metadata2.merge(breast_anno2, on=['image_id'], how='inner')
train.to_csv('input/rsna-breast-cancer-detection/vindr_train.csv', index=False)

In [66]:
finding_anno2.merge(breast_anno2[['image_id', 'patient_id']], on='image_id', how='inner').to_csv('input/rsna-breast-cancer-detection/vindr_findings.csv', index=False)

# Vindr

In [3]:
train_vindr = pd.read_csv('input/rsna-breast-cancer-detection/vindr_train.csv')
train = pd.read_csv('input/rsna-breast-cancer-detection/train.csv')

In [4]:
train_vindr['BIRADS'].value_counts()

1    13406
2     4676
3      930
4      762
5      226
Name: BIRADS, dtype: int64

In [5]:
train_vindr['cancer'] = (train_vindr['BIRADS'] >= 4).astype(int)

In [7]:
train_vindr.to_csv('input/rsna-breast-cancer-detection/vindr_train.csv', index=False)

In [11]:
train_vindr.query('BIRADS.isin([1, 5])').to_csv('input/rsna-breast-cancer-detection/vindr_train_birads05.csv', index=False)

In [19]:
pd.concat([train, train_vindr.query('BIRADS.isin([1, 5])')]).drop(['view.1', 'is_test'], axis=1).to_csv(
    'input/rsna-breast-cancer-detection/train_concat_vindr_birads05.csv', index=False)

In [20]:
pd.concat([train, train_vindr.query('BIRADS.isin([1, 5])')]).drop(['view.1', 'is_test'], axis=1).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68338 entries, 0 to 19994
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   site_id                  54706 non-null  float64
 1   patient_id               68338 non-null  object 
 2   image_id                 68338 non-null  object 
 3   laterality               68338 non-null  object 
 4   view                     68338 non-null  object 
 5   age                      66797 non-null  float64
 6   cancer                   68338 non-null  int64  
 7   biopsy                   54706 non-null  float64
 8   invasive                 54706 non-null  float64
 9   BIRADS                   39918 non-null  float64
 10  implant                  54706 non-null  float64
 11  density                  43102 non-null  object 
 12  machine_id               68338 non-null  object 
 13  difficult_negative_case  54706 non-null  object 
dtypes: float64(6), int64(1